In [ ]:
import requests
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Import API key
from config import key

Realizing that every public datafile contains either the name, or the FIPS code of a place, we first downloaded the FIPS codes csv from census.gov

In [2]:
state_county = pd.read_csv("all-geocodes-v2017.csv", dtype={'County Code (FIPS)': object, 'State Code (FIPS)': object})
state_county

state_county.columns


Index(['Summary Level', 'State Code (FIPS)', 'County Code (FIPS)',
       'County Subdivision Code (FIPS)', 'Place Code (FIPS)',
       'Consolidtated City Code (FIPS)',
       'Area Name (including legal/statistical area description)'],
      dtype='object')

The full file is a bit too granular for our purposes so we cleaned it up to only include FIPS codes for states and counties.

In [3]:
state_county_clean = state_county.loc[(state_county['County Subdivision Code (FIPS)'] == 0) & 
                                      (state_county['Place Code (FIPS)'] == 0) & 
                                      (state_county['Consolidtated City Code (FIPS)'] == 0)]

state_county_clean.head()


,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,40,01,000,0,0,0,Alabama
1,50,01,001,0,0,0,Autauga County
2,50,01,003,0,0,0,Baldwin County
3,50,01,005,0,0,0,Barbour County
4,50,01,007,0,0,0,Bibb County


In [4]:
state_clean = state_county.loc[(state_county['County Code (FIPS)'] == '000') & 
                               (state_county['County Subdivision Code (FIPS)'] == 0) & 
                               (state_county['Place Code (FIPS)'] == 0) & 
                               (state_county['Consolidtated City Code (FIPS)'] == 0)]

state_clean.head()


,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,40,01,000,0,0,0,Alabama
529,40,02,000,0,0,0,Alaska
707,40,04,000,0,0,0,Arizona
814,40,05,000,0,0,0,Arkansas
1391,40,06,000,0,0,0,California


In [5]:
county_clean = state_county.loc[(state_county['County Code (FIPS)'] != '000') & 
                                (state_county['County Subdivision Code (FIPS)'] == 0) & 
                                (state_county['Place Code (FIPS)'] == 0) & 
                                (state_county['Consolidtated City Code (FIPS)'] == 0)]

county_clean.head()


,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
1,50,01,001,0,0,0,Autauga County
2,50,01,003,0,0,0,Baldwin County
3,50,01,005,0,0,0,Barbour County
4,50,01,007,0,0,0,Bibb County
5,50,01,009,0,0,0,Blount County


I then pulled the income data using the Census API. From the documentation I found out that the median household income variable is coded "DP03_0062E," and the names can be pulled along with the variable if specified in the API URL.

In [ ]:
response_list = []

for index, row in county_clean.iterrows():
    state_code = row['State Code (FIPS)']
    county_code = row['County Code (FIPS)']
    
    print(state_code)
    print(county_code)
    
    url = f"https://api.census.gov/data/2018/acs/acs5/profile?get=DP03_0062E,NAME&for=county:{county_code}&in=state:{state_code}&key={key}"
   
    try:
        r = requests.get(url)
        response = requests.get(url)
        data = response.json()
        
        index = data[0]
        value = data[1]
        
        df_dict = {'Index':index,'Value':value}
        response_list.append(df_dict)
        
    except Exception as e:
        print(e)
        
        
response_list



01
001
01
003
01
005
01
007
01
009
01
011
01
013
01
015
01
017
01
019
01
021
01
023
01
025
01
027
01
029
01
031
01
033
01
035
01
037
01
039
01
041
01
043
01
045
01
047
01
049
01
051
01
053
01
055
01
057
01
059
01
061
01
063
01
065
01
067
01
069
01
071
01
073
01
075
01
077
01
079
01
081
01
083
01
085
01
087
01
089
01
091
01
093
01
095
01
097
01
099
01
101
01
103
01
105
01
107
01
109
01
111
01
113
01
115
01
117
01
119
01
121
01
123
01
125
01
127
01
129
01
131
01
133
02
013
02
016
02
020
02
050
02
060
02
068
02
070
02
090
02
100
02
105
02
110
02
122
02
130
02
150
02
158
02
164
02
170
02
180
02
185
02
188
02
195
02
198
02
220
02
230
02
240
02
261
02
275
02
282
02
290
04
001
04
003
04
005
04
007
04
009
04
011
04
012
04
013
04
015
04
017
04
019
04
021
04
023
04
025
04
027
05
001
05
003
05
005
05
007
05
009
05
011
05
013
05
015
05
017
05
019
05
021
05
023
05
025
05
027
05
029
05
031
05
033
05
035
05
037
05
039
05
041
05
043
05
045
05
047
05
049
05
051
05
053
05
055
05
057
05
059
05
061
05
063

For safety purposes (and this presentation), I also downloaded the county-level information from the census website, which can be loaded more quickly. This file had A LOT of information, from which I extracted median household income estimates (instead of median household income margin of error, or percent estimate, or percent estimate margin of error)

In [ ]:
income_counties_df = pd.read_csv("median_hhi_counties.csv")

income_counties_df.columns = income_counties_df.iloc[0]

income_counties_df = income_counties_df.iloc[1:]

income_counties_df = income_counties_df.set_index('id')

income_counties_df = income_counties_df.filter(regex="Median household income")

income_counties_df = income_counties_df.filter(regex="Estimate!!")

income_counties_df = income_counties_df.reset_index()

income_counties_df.columns = ["id", "Median HHI", "Median HHI Perc"]

income_counties_df = income_counties_df[['id', 'Median HHI']]

income_counties_df


In [ ]:
income_counties_df['state_code'] = income_counties_df.id.str[9:11]
income_counties_df


In [ ]:
income_counties_df['county_code'] = income_counties_df.id.str[11:14]
income_counties_df


In [ ]:
income_counties_df = income_counties_df.astype({"state_code": int, "county_code": int})
income_counties_df.to_csv("median_hhi_counties.csv")

income_counties_df

From my previous group project, I already had a state-level income file (credits to Julia Leonoff), which I cleaned up to contain the most-recent information for queries that need state-level income only.

In [ ]:
income_df = pd.read_csv("https://raw.githubusercontent.com/gkmatt29/ETL-Enthusiasts/master/Resources/household_median_income_2017.csv")

median_hhi_2017_state = income_df[["State", "2017"]]

median_hhi_2017_state.columns = ['State', 'Median HHI']

median_hhi_2017_state["Year"] = 2017

median_hhi_2017_state

median_hhi_2017_state.to_csv('median_hhi_2017_state.csv')
